In [ ]:
!pip install pandas scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/"

movies = pd.read_csv(path + "title.basics.tsv", sep="\t", low_memory=False)
ratings = pd.read_csv(path + "title.ratings.tsv", sep="\t")


In [ ]:
movies = movies[movies["titleType"] == "movie"]
movies = movies[["tconst", "primaryTitle", "genres"]]


In [ ]:
movies = movies[movies["genres"] != "\\N"]
movies["genres"] = movies["genres"].str.replace(",", " ")


In [ ]:
movies = movies.merge(ratings, on="tconst")


In [ ]:
movies = movies[movies["numVotes"] > 10000]
movies = movies.reset_index(drop=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["genres"])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)


In [ ]:
def recommend(movie_title, top_n=5):
    if movie_title not in movies["primaryTitle"].values:
        return "Movie not found"

    idx = movies[movies["primaryTitle"] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    return movies.iloc[[i[0] for i in sim_scores]][
        ["primaryTitle", "genres", "averageRating"]
    ]


In [ ]:
recommend("The Conjuring", 10)


,primaryTitle,genres,averageRating
890,The Cat o' Nine Tails,Horror Mystery Thriller,6.6
947,A Bay of Blood,Horror Mystery Thriller,6.5
990,Don't Torture a Duckling,Horror Mystery Thriller,7.0
1038,Sisters,Horror Mystery Thriller,6.8
1048,The Wicker Man,Horror Mystery Thriller,7.5
1054,Black Christmas,Horror Mystery Thriller,7.1
1121,Deep Red,Horror Mystery Thriller,7.5
1141,Burnt Offerings,Horror Mystery Thriller,6.4
1331,When a Stranger Calls,Horror Mystery Thriller,6.4
1366,Friday the 13th,Horror Mystery Thriller,6.4
